In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
from pathlib import Path
import re
from typing import Any
from pydantic import Field

from grasp_agents.grasp_logging import setup_logging
from grasp_agents.agent_message_pool import (
    AgentMessagePool,
)
from grasp_agents.llm_agent import LLMAgent
from grasp_agents.openai.openai_llm import (
    OpenAILLM,
    OpenAILLMSettings,
)
from grasp_agents.typing.io import (
    AgentPayload,
    LLMPromptArgs,
)
from pydantic import BaseModel
from grasp_agents.typing.tool import BaseTool
from grasp_agents.run_context import RunContextWrapper, RunArgs
from grasp_agents.agent_message import AgentMessage
from grasp_agents.typing.content import ImageData
from grasp_agents.utils import get_timestamp
from grasp_agents.workflow.sequential_agent import SequentialWorkflowAgent
from grasp_agents.llm_agent_state import LLMAgentState

In [ ]:
PACKAGE_DIR = Path.cwd()
DATA_DIR = Path.cwd() / "data/multiagent"

In [ ]:
LOGGING_DIR = DATA_DIR / "logs"
LOGGING_CFG_PATH = PACKAGE_DIR / "configs/logging/default.yaml"
setup_logging(
    LOGGING_DIR / f"agents_demo_{get_timestamp()}.log",
    LOGGING_CFG_PATH,
)

# Chat

In [ ]:
IMG_1_URL = "https://www.simplilearn.com/ice9/free_resources_article_thumb/Expressions_In_C_2.PNG"
IMG_2_PATH = Path.cwd() / "src/grasp_agents/examples/data/expr.jpeg"

In [ ]:
class Response(AgentPayload):
    response: str


chatbot = LLMAgent[Any, Response, None](
    agent_id="chatbot",
    llm=OpenAILLM(
        model_name="gpt-4.1",
        llm_settings=OpenAILLMSettings(),
    ),
    sys_prompt=None,
    out_schema=Response,
)


@chatbot.parse_output_handler
def output_handler(conversation, ctx, **kwargs) -> Response:
    return Response(response=conversation[-1].content)


# This initialises printer and usege tracker
ctx = RunContextWrapper(print_messages=True)

In [ ]:
out = await chatbot.run("Where are you headed, stranger?", ctx=ctx)
print(out.payloads[0].response)

In [ ]:
out = await chatbot.run("What did you just say, exactly?", ctx=ctx)
print(out.payloads[0].response)

In [ ]:
out = await chatbot.run(
    ["What's in this image?", ImageData.from_path(IMG_2_PATH)], ctx=ctx
)
print(out.payloads[0].response)

In [ ]:
out = await chatbot.run("Go on", ctx=ctx)
print(out.payloads[0].response)

In [ ]:
out = await chatbot.run(["Try another one", ImageData.from_url(IMG_1_URL)], ctx=ctx)
print(out.payloads[0].response)

In [ ]:
out = await chatbot.run("What was my first question, exactly?", ctx=ctx)
print(out.payloads[0].response)

In [ ]:
ctx.usage_tracker.total_usage

# Structured outputs

In [ ]:
from enum import StrEnum


class Selector(StrEnum):
    S1 = "S1"
    S2 = "S2"


class Response(AgentPayload):
    a: int = Field(..., description="This integer must be equal to 1123")
    b: bool = Field(..., description="This boolean must be equal to False")
    s: Selector = Field(..., description="Choose a value randomly")

In [ ]:
# Structured outputs are not supported via openrouter, but generated messages
# can still be validated against the provided schema (response_format)

# In case structured outputs are not supported, we need to provide the output schema
# in the prompt. Otherwise the prompt can be empty in this example.

# inp_prompt = """
# Output a plain JSON string (no formatting) that contains the following fields:
# - a: an integer that must be equal to 1123
# - b: a boolean that must be equal to False
# - s: a string that must be either "S1" or "S2"

# {{
#   "a": <integer>,
#   "b": <boolean>,
#   "s": <string>
# }}
# """

test_agent = LLMAgent[Any, Response, None](
    agent_id="test_agent",
    llm=OpenAILLM(
        model_name="gemini-2.5-pro-preview-03-25",
        api_provider="google_ai_studio",
        # model_name="gpt-4.1",
        # api_provider="openai",
        # model_name="anthropic/claude-3.7-sonnet",
        # api_provider="openrouter",
        num_generation_retries=0,
        llm_settings=OpenAILLMSettings(temperature=0),
        response_format=Response,
    ),
    inp_prompt="",
    out_schema=Response,
)

In [ ]:
ctx = RunContextWrapper(print_messages=True)
out = await test_agent.run(ctx=ctx)
out.payloads[0]

# Simple batching

In [ ]:
sys_prompt = "You are a bad math student who always adds number {added_num} to the correct result of the operation."
usr_prompt = "What is the square of {num}?"


class SystemArgs(LLMPromptArgs):
    added_num: int


class InputArgs(LLMPromptArgs):
    num: int


class Response(AgentPayload):
    response: str


student = LLMAgent[Any, Response, None](
    agent_id="student",
    llm=OpenAILLM(
        model_name="gpt-4.1",
        llm_settings=OpenAILLMSettings(),
        # rate_limiter_rpm=50,
    ),
    sys_prompt=sys_prompt,
    sys_args_schema=SystemArgs,
    inp_prompt=usr_prompt,
    usr_args_schema=InputArgs,
    out_schema=Response,
    set_state_strategy="keep",
)


@student.parse_output_handler
def output_handler(conversation, ctx, **kwargs) -> Response:
    return Response(response=conversation[-1].content)

#### One system prompt -> many user arguments

In [ ]:
run_args = RunArgs(
    sys=SystemArgs(added_num=1),
    usr=[InputArgs(num=i) for i in range(1, 10)],
)

ctx = RunContextWrapper(run_args={"student": run_args}, print_messages=True)

In [ ]:
out = await student.run(ctx=ctx)
print(*[p.response for p in out.payloads], sep="\n")

#### Many back to one

Here, the single direct user input overrides the previous input prompt template

In [ ]:
out = await student.run(
    "Who are you, dear stranger? What was your last chore?", ctx=ctx
)
print(*[p.response for p in out.payloads], sep="\n")

In [ ]:
ctx.usage_tracker.total_usage

# ReAct agent loop 

In [ ]:
sys_prompt_react = """
You are a gifted stats tutor. Your task is to suggest an exciting stats problem to the student. 
You should first ask the student about their education, interests, and preferences, then suggest a problem tailored specifically to them. 

# Instructions
* Ask questions one by one.
* Provide your thinking before asking a question and after receiving a reply.
* Do not include your exact question as part of your thinking.
* The problem must have all the necessary data.
* The problem must be enclosed in <PROBLEM> tags.
"""

In [ ]:
class TeacherQuestion(BaseModel):
    question: str


class StudentReply(BaseModel):
    reply: str


ask_student_tool_description = """
"Ask the student a question and get their reply."

Args:
    question: str
        The question to ask the student.
Returns:
    {"reply": str}
        Dictionary containing the student's reply to the question.
"""


class AskStudentTool(BaseTool[TeacherQuestion, StudentReply, Any]):
    name: str = "ask_student"
    description: str = ask_student_tool_description
    in_schema: type[TeacherQuestion] = TeacherQuestion
    out_schema: type[StudentReply] = StudentReply

    async def run(
        self, inp: BaseModel, ctx: RunContextWrapper[Any] | None = None
    ) -> StudentReply:
        reply = input(inp.question)

        return StudentReply(reply=reply)

In [ ]:
class Response(AgentPayload):
    problem: str


teacher = LLMAgent[Any, Response, None](
    agent_id="teacher",
    llm=OpenAILLM(
        model_name="gpt-4.1",
        api_provider="openai",
        llm_settings=OpenAILLMSettings(temperature=0.5),
        # rate_limiter_rpm=50,
    ),
    tools=[AskStudentTool()],
    max_turns=20,
    react_mode=True,
    sys_prompt=sys_prompt_react,
    out_schema=Response,
    set_state_strategy="reset",
)


@teacher.tool_call_loop_exit_handler
def tool_call_loop_exit(conversation, ctx, **kwargs) -> None:
    message = conversation[-1].content

    return re.search(r"<PROBLEM>", message)


@teacher.parse_output_handler
def parse_output(conversation, ctx, **kwargs) -> Response:
    return Response(problem=conversation[-1].content)

In [ ]:
ctx = RunContextWrapper(print_messages=True)

In [ ]:
out = await teacher.run(ctx=ctx)
print(out.payloads[0].problem)

# Sequential workflow 

In [ ]:
add_inp_prompt = "Add {a} and {b}. Your only output is the resulting number."


# Received arguments are passed to the agent dynamically
class AddReceivedArgs(AgentPayload):
    a: int = Field(..., description="First number to add.")


# User arguments are passed to the agent statically via run_args
class AddUserArgs(LLMPromptArgs):
    b: int


class AddResponse(AgentPayload):
    result: int


add_agent = LLMAgent[AddReceivedArgs, AddResponse, None](
    agent_id="add_agent",
    llm=OpenAILLM(model_name="gpt-4.1", llm_settings=OpenAILLMSettings()),
    rcv_args_schema=AddReceivedArgs,
    usr_args_schema=AddUserArgs,
    inp_prompt=add_inp_prompt,
    out_schema=AddResponse,
    set_state_strategy="reset",
)


@add_agent.format_inp_args_handler
def format_inp_args(usr_args: AddUserArgs, rcv_args: AddReceivedArgs, ctx):
    return {"a": rcv_args.a, "b": usr_args.b}


@add_agent.parse_output_handler
def output_handler(conversation, rcv_args, ctx) -> AddResponse:
    return AddResponse(result=int(conversation[-1].content))

In [ ]:
class MultiplyUserArgs(LLMPromptArgs):
    c: int


class MultiplyResponse(AgentPayload):
    result: int


multiply_inp_prompt = (
    "Multiply {inp} and {c}. Your only output is the resulting number."
)

multiply_agent = LLMAgent[AddResponse, MultiplyResponse, None](
    agent_id="multiply_agent",
    llm=OpenAILLM(model_name="gpt-4.1", llm_settings=OpenAILLMSettings()),
    rcv_args_schema=AddResponse,
    usr_args_schema=MultiplyUserArgs,
    inp_prompt=multiply_inp_prompt,
    out_schema=MultiplyResponse,
    set_state_strategy="reset",
)


@multiply_agent.format_inp_args_handler
def format_inp_args(
    usr_args: MultiplyUserArgs, rcv_args: AddResponse, ctx
) -> dict[str, str]:
    # Combine the output of the add agent with the user input for multiplication
    return {"inp": rcv_args.result, "c": usr_args.c}


@multiply_agent.parse_output_handler
def output_handler(conversation, rcv_args, ctx) -> MultiplyResponse:
    return MultiplyResponse(result=int(conversation[-1].content))

In [ ]:
seq_agent = SequentialWorkflowAgent[AddUserArgs, MultiplyResponse, None](
    subagents=[add_agent, multiply_agent],
    agent_id="seq_agent",
    out_schema=MultiplyResponse,
)

In [ ]:
# Can use batches of inputs here as well
add_run_args = RunArgs(usr=AddUserArgs(b=3))
multiply_run_args = RunArgs(usr=MultiplyUserArgs(c=5))

In [ ]:
ctx = RunContextWrapper(
    run_args={"add_agent": add_run_args, "multiply_agent": multiply_run_args},
    print_messages=True,
)

In [ ]:
rcv_message = AgentMessage(payloads=[AddReceivedArgs(a=2)], sender_id="user")

In [ ]:
out = await seq_agent.run(rcv_message=rcv_message, ctx=ctx)
print(out.payloads[0].result)

# Agents as tools

When used as tools, the tool inputs are rcv_args

In [ ]:
seq_tool = seq_agent.as_tool(
    tool_name="seq_agent_tool",
    tool_description=(
        "A sequential agent that adds 3 to a given integer, "
        "then multiplies the result by 5.",
    ),
)

Description of rcv_args is preserved

In [ ]:
seq_tool.in_schema.model_json_schema()

In [ ]:
await seq_tool(a=15, ctx=ctx)

# Teacher / students

In [ ]:
def extract_recipients(message: str) -> list[str]:
    # Find the substring that matches the pattern inside brackets with angle brackets
    match = re.search(r"\[(.*?)\]", message)

    if match:
        # Extract the contents inside square brackets
        content = match.group(1)

        # Extract each student name within angle brackets
        students = re.findall(r"<(.*?)>", content)

        return students  # Output: ['Alice', 'Bob', 'Charlie']
    return []

#### Communication schemas

In [ ]:
class TeacherExplanation(AgentPayload):
    explanation: str


class StudentQuestion(AgentPayload):
    question: str

In [ ]:
pool = AgentMessagePool()

#### Teacher

In [ ]:
teacher_sys_prompt = """
You are a teacher explaining quantum gravity to a 2-year old baby (named student1) and a 30-year old graphic designer (named student2). 
Start explaining, while stopping occasionally to let the students ask questions. 
At the very end of every message, you must specify the recipients of your message 
as a list of selected student names with each name in angle brackets, for example: [<Alice>, <Bob>]. 
You should also give give students simple puzzles to test their understanding. 
Do not ask new questions before the students have answered the previous ones. 
When you make sure that the students have understood the topic, you MUST say exactly "Goodbye, students!" and terminate the conversation.
"""

teacher = LLMAgent[StudentQuestion, TeacherExplanation, None](
    agent_id="teacher",
    llm=OpenAILLM(
        model_name="gpt-4o",
        llm_settings=OpenAILLMSettings(),
    ),
    sys_prompt=teacher_sys_prompt,
    rcv_args_schema=StudentQuestion,
    out_schema=TeacherExplanation,
    message_pool=pool,
    recipient_ids=["student1", "student2"],
    set_state_strategy="keep",
    dynamic_routing=True,
)


@teacher.parse_output_handler
def teacher_output_handler(conversation, rcv_args, ctx) -> TeacherExplanation:
    message = conversation[-1].content
    recipients = extract_recipients(message)
    explanation = message.split("[")[0].strip()

    return TeacherExplanation(
        explanation=explanation, selected_recipient_ids=recipients
    )


@teacher.exit_handler
def teacher_exit_handler(
    teacher_out: AgentMessage, ctx: RunContextWrapper | None = None
) -> None:
    message = teacher_out.payloads[0].explanation
    return re.search(r"Goodbye, students!", message)

#### Students

In [ ]:
student_sys_prompts = [
    """
You are a 2-year old baby trying to make sense of physics. 
Your name is <student1>.
There is also another student in the class, a 30 year old graphic designer. 
You talk to the teacher only.
""",
    """
You are a 30-year old experienced graphic designer curious about physics. 
Your name is <student2>.
Ask questions to the teacher until you understand the topic fully. 
Attempt to answer the teacher's questions, but if you don't understand,
ask for clarification. 
There is also another student in the class, a 2 year old baby.
""",
]


def make_student_agent(name: str, sys_prompt: str):
    return LLMAgent[TeacherExplanation, StudentQuestion, None](
        agent_id=name,
        llm=OpenAILLM(
            model_name="gpt-4o",
            llm_settings=OpenAILLMSettings(),
        ),
        sys_prompt=sys_prompt,
        rcv_args_schema=TeacherExplanation,
        out_schema=StudentQuestion,
        message_pool=pool,
        recipient_ids=["teacher"],
        set_state_strategy="keep",
        dynamic_routing=False,
    )


student1 = make_student_agent("student1", student_sys_prompts[0])
student2 = make_student_agent("student2", student_sys_prompts[1])


@student1.parse_output_handler
def student1_output_handler(conversation, rcv_args, ctx) -> StudentQuestion:
    return StudentQuestion(question="<student1>: " + conversation[-1].content)


@student2.parse_output_handler
def student2_output_handler(conversation, rcv_args, ctx) -> StudentQuestion:
    return StudentQuestion(question="<student2>: " + conversation[-1].content)

#### Specify context to be shared across 

In [ ]:
ctx = RunContextWrapper(print_messages=True)
ctx.printer.color_by = "agent_id"

#### Run

Wait until completion. 

There might be some message pool exceptions at the end that don't affect the functionality.

In [ ]:
await teacher.start_listening(ctx=ctx)
await student1.start_listening(ctx=ctx)
await student2.start_listening(ctx=ctx)
await teacher.run_and_post(ctx=ctx)